In [ ]:
import ray
ray.shutdown()

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
from qgrid import show_grid
# import finplot as fplt

import polygon_rest_api as papi
import polygon_ws_api as pws
import polygon_df as pdf
import polygon_ds as pds
import polygon_s3 as ps3

import bar_samples as bs
import bar_labels as bl
import bar_meta as bm
import filters as ft

import flow_backfill as fb

# import market_cluster as mc

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# .plot_bokeh(sizing_mode="scale_height")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# symbol='GLD'
symbol = 'market'
date = '2020-01-02'
start_date = '2020-10-01'
end_date = '2020-10-10'
# tick_type='trades'
tick_type='daily'

thresh = {
    # max thresholds
    'renko_reveral_multiple': 2,
    'max_duration_sec': 60*60,
    # min thresholds
    'min_duration_sec': 15,
    'min_tick_count': 30,
}

horizon_mins=30

In [19]:
mdf = ps3.load_date_df(symbol, date, tick_type)

market 2020-01-02 trying to get ticks from local file...


In [21]:
df

,sequence,trade_id,exchange_id,price,size,conditions,irregular,sip_dt,exchange_dt
0,28701,52983525027910,11,142.940002,6,"[12, 37]",False,2020-01-02 09:00:40.834409467,2020-01-02 09:00:40.833902336
1,29001,3472338191832601422,12,143.130005,10,"[12, 37]",False,2020-01-02 09:15:34.068353536,2020-01-02 09:15:34.067539269
2,29801,3472338191832602726,12,143.169998,754,"[14, 12, 41]",False,2020-01-02 09:27:21.328832780,2020-01-02 09:27:21.328049503
3,30001,52983525028144,11,143.179993,100,"[14, 12, 41]",False,2020-01-02 09:27:34.171401700,2020-01-02 09:27:34.171080704
4,30101,3472338191832602988,12,143.179993,100,"[14, 12, 41]",False,2020-01-02 09:29:19.062858164,2020-01-02 09:29:19.062070226
...,...,...,...,...,...,...,...,...,...
31046,40196001,52983525624633,8,144.250000,67,"[14, 12, 37, 41]",False,2020-01-03 00:58:04.441735786,2020-01-03 00:58:04.441173000
31047,40196401,71684468919122,4,144.000000,10,"[12, 37]",False,2020-01-03 00:58:35.385387039,2020-01-03 00:58:35.381000000
31048,40196601,52983525624638,8,144.009995,10,"[12, 37, 41]",False,2020-01-03 00:59:34.154720166,2020-01-03 00:59:34.154056000
31049,40199001,52983559834699,11,143.949997,0,"[38, 41]",True,2020-01-03 01:00:00.002102500,2020-01-03 01:00:00.001874688


In [11]:
ps3.list_symbol_dates(symbol)

['2020-01-02']

In [ ]:
df = ps3.load_ticks(symbol, '2020-11-05')
sdf = df.loc[df.irregular==False]
sdf2 = ft.jma_filter_df(sdf, 'price', length=5, power=0.5)

In [ ]:
(abs(sdf2.price - sdf2.price_jma) / sdf2.price_jma).describe(percentiles=[.99,.999,.9995,.9999])

In [ ]:
fb.run_backfill(symbols=['GLD'], tick_type='trades', start_date='2020-10-01')
# fb.run_backfill(symbols=['market'], tick_type='daily', start_date='2020-10-01')

In [ ]:
import ray
ray.init(ignore_reinit_error=True, num_cpus=4, dashboard_port=8080)

daily_bar_stats_df, labeled_bar_dates, stacked_bars_df = bm.bars_workflow_ray( 
    symbol, 
    start_date, 
    end_date, 
    thresh,
    imbalance_thresh=0.9
)

In [ ]:
import pickle 
fio = open('bars_workflow_ray-new.pickle', 'wb') 
pickle.dump((daily_bar_stats_df, labeled_bar_dates, stacked_bars_df), fio)

In [ ]:
data = (daily_bar_stats_df, labeled_bar_dates, stacked_bars_df)
file = open('test.pickle', mode='wb')
pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("test.pickle", "rb") as input_file:
    e = pickle.load(input_file)

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])